In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
import re

from PIL import Image

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from albumentations.pytorch.transforms import ToTensorV2, ToTensor

import torch
import torchvision

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler

from matplotlib import pyplot as plt

In [ ]:
train_df=pd.read_csv('../input/global-wheat-detection/train.csv')

In [ ]:
df=train_df

In [ ]:
df=df[['image_id', 'source']].drop_duplicates()


In [ ]:
s_map={'arvalis_1':1,'ethz_1':2,'arvalis_3':3,'rres_1':4,'arvalis_2':5,'usask_1':6,'inrae_1':7,}

In [ ]:
df['mapping']=df.source.map(s_map)

In [ ]:
h=[i for i in range(len(df))]

In [ ]:
df.set_index(pd.Index(h), inplace = True) 

In [ ]:
df['folds']=-1

In [ ]:
from sklearn import model_selection

In [ ]:
kf = model_selection.StratifiedKFold(n_splits=5)


In [ ]:
y=df.mapping.values

In [ ]:
for f, (t_, v_) in enumerate(kf.split(X=df, y=y)):
    df.loc[v_, 'folds'] = f

In [ ]:
train_df['folds']=-1

In [ ]:
mapp=df.groupby(['image_id']).folds.mean()
train_df.folds=train_df.image_id.map(mapp)

In [ ]:
import re
train_df['x'] = -1
train_df['y'] = -1
train_df['w'] = -1
train_df['h'] = -1

def expand_bbox(x):
    r = np.array(re.findall("([0-9]+[.]?[0-9]*)", x))
    if len(r) == 0:
        r = [-1, -1, -1, -1]
    return r

train_df[['x', 'y', 'w', 'h']] = np.stack(train_df['bbox'].apply(lambda x: expand_bbox(x)))
train_df.drop(columns=['bbox'], inplace=True)
train_df['x'] = train_df['x'].astype(np.float)
train_df['y'] = train_df['y'].astype(np.float)
train_df['w'] = train_df['w'].astype(np.float)
train_df['h'] = train_df['h'].astype(np.float)

In [ ]:
dff=train_df
dff['area']=train_df.w*train_df.h
dff.head()

In [ ]:
filt2=(dff.area<85000)
dff=dff[filt2]

In [ ]:
filt1=(dff.area>150)
dff=dff[filt1]


In [ ]:
dff['x_min']=dff.x
dff['y_min']=dff.y
dff['x_max']=dff.x+dff.w
dff['y_max']=dff.y+dff.h


In [ ]:
train_df = dff[['image_id', 'x_min', 'y_min', 'x_max', 'y_max', 'width', 'height', 'area', 'source','folds']]
train_df.head()

In [ ]:
def get_train_transform():
    return A.Compose(
        [
            A.RandomSizedCrop(min_max_height=(700, 700), height=1024, width=1024, p=0.3),
            
            
           A.OneOf([A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit= 0.2, 
                                     val_shift_limit=0.2, p=0.5),
                    A.RandomBrightnessContrast(brightness_limit=0.2, 
                                           contrast_limit=0.2, p=0.5)],p=0.75),
            A.GaussNoise(0.3,p=0.2),
            A.RandomShadow(p=0.3),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            
            A.Cutout(num_holes=4, max_h_size=100, max_w_size=100, fill_value=[0.6,0.6,0.5], p=0.35),
            ToTensorV2(p=1.0),
        ],p=1.0,
        bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels']
        )
    )

def get_valid_transform():
    return A.Compose(
        [
            
            ToTensorV2(p=1.0),
        ], 
        p=1.0, 
        bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels']
        )
    )

In [ ]:
class WheatDataset(Dataset):

    def __init__(self, df, image_dir,folds,transforms=None):
        super().__init__()
        self.df = df[df.folds.isin(folds)].reset_index(drop=True)
        self.image_ids = self.df['image_id'].unique()
        self.image_dir = image_dir
        self.transforms = transforms

    def __getitem__(self, index: int):

        image_id = self.image_ids[index]
        records = self.df[self.df['image_id'] == image_id]
        
            
        image = cv2.imread(f'{self.image_dir}/{image_id}.jpg', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
            

        boxes = records[['x_min', 'y_min', 'x_max', 'y_max']].values
        
        
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        area = torch.as_tensor(area, dtype=torch.float32)

        # there is only one class
        labels = torch.ones((records.shape[0],), dtype=torch.int64)
        
        # suppose all instances are not crowd
        iscrowd = torch.zeros((records.shape[0],), dtype=torch.int64)
        
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = torch.tensor([index])
        target['area'] = area
        target['iscrowd'] = iscrowd

        if self.transforms:
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': labels
            }
            sample = self.transforms(**sample)
            image = sample['image']
            
            if (len(sample['bboxes'])>0):
                target['boxes']=torch.as_tensor(sample['bboxes'],dtype=torch.float32)
            else:
                target['boxes']= torch.linspace(0,3,steps=4,dtype=torch.float32)
                target['boxes']=target['boxes'].reshape(-1,4)

        return image, target, image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]

In [ ]:
DIR_INPUT = '/kaggle/input/global-wheat-detection'
DIR_TRAIN = f'{DIR_INPUT}/train'

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))
train_dataset = WheatDataset(train_df,DIR_TRAIN,[1, 2, 3, 4],get_train_transform())
indices = torch.randperm(len(train_dataset)).tolist()

train_data_loaders = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
    num_workers=0,
    collate_fn=collate_fn
)

In [ ]:
images, targets, image_ids = next(iter(train_data_loaders))
images = list(image.to(torch.device('cpu')) for image in images)
targets = [{k: v.to(torch.device('cpu')) for k, v in t.items()} for t in targets]
boxes = targets[0]['boxes'].cpu().numpy().astype(np.int32)
sample = images[0].permute(1,2,0).cpu().numpy()
fig, ax = plt.subplots(1, 1, figsize=(16, 8))

for box in boxes:
    cv2.rectangle(sample,
                  (box[0], box[1]),
                  (box[2], box[3]),
                  (220, 0, 0), 3)
    
ax.set_axis_off()
ax.imshow(sample)

In [ ]:
DIR_WEIGHTS = '.../input/base-512x512'

WEIGHTS_FILE = '../input/base-512x512/base.pth'


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

In [ ]:
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
backbone = resnet_fpn_backbone('resnet50',False)
model = FasterRCNN(backbone, num_classes=2)

In [ ]:
num_classes = 2  # 1 class (wheat) + background

# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features

# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# Load the trained weights
model.load_state_dict(torch.load(WEIGHTS_FILE))
model.to(device)

In [ ]:
class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0

In [ ]:
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=0.00001)
#lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
#lr_scheduler = None

num_epochs = 17

In [ ]:
# CUSTOM LEARNING SCHEUDLE
LR_START = 1e-5
LR_MAX = 1e-3
LR_RAMPUP_EPOCHS = 2
LR_SUSTAIN_EPOCHS = 0
LR_STEP_DECAY = 0.75

def lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = LR_MAX * LR_STEP_DECAY**((epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS)//2)
    return lr
lr_scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lrfn)

In [ ]:
loss_hist = Averager()
itr = 1

for epoch in range(num_epochs):
    loss_hist.reset()
    
    for images, targets, image_ids in train_data_loaders:
        
        images = list(image.to(device) for image in images)
        targets= [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)

        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()

        loss_hist.send(loss_value)

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if itr % 70 == 0:
            print(f"Iteration #{itr} loss: {loss_value}")

        itr += 1
    
    # update the learning rate
    if lr_scheduler is not None:
        lr_scheduler.step()

    print(f"Epoch #{epoch} loss: {loss_hist.value}") 

In [ ]:
torch.save(model.state_dict(), 'model0.pth')